In [9]:
git_dir / "CORDEX_eval_scripts/catalog.csv"

PosixPath('/dodrio/scratch/projects/2022_200/project_output/RMIB-UGent/vsc31332_inne/ValEnsPy/CORDEX_eval_scripts/catalog.csv')

In [ ]:
import xarray as xr
import valenspy as vp
from datatree import DataTree
from pathlib import Path
import pandas as pd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import os

#Get the git directory using the Path object using an os command
git_dir = Path(os.popen("git rev-parse --show-toplevel").read().strip())


#User options
variables = ["tas", "pr"]
period = [1980,2019]
target_grid="/dodrio/scratch/projects/2022_200/external/climate_grid/TEMP_AVG_CLIMATE_GRID_1954_2023_daily.nc"

############################################
# STEP 1: Loading the data

# start input manager
manager = vp.InputManager(machine="hortense")


#MODEL data
# Load the ALARO data

df_alaro = pd.read_csv(git_dir / "CORDEX_eval_scripts/catalog.csv")
df_alaro = df_alaro[df_alaro['frequency'] == 'day']
df_alaro = df_alaro[df_alaro['variable_id'].isin(variables)]
df_alaro

#Load all the paths in the df into one xarray dataset
ds_alaro = xr.open_mfdataset(df_alaro['path'].values, decode_coords='all', chunks="auto")
ds_alaro

# Load the COSMO data
## Issue - variarble names are hardcoded due to daily statistic, necessary to find correct path with input manager
experiment = "CB2_CCLM_EUR11_ERA5_evaluation"
ds_cclm_tas = manager.load_data("CCLM", ["tas"], freq="daily", path_identifiers=[experiment, "mean"])
ds_cclm_pr  = manager.load_data("CCLM", ["pr"], freq="daily", path_identifiers=[experiment, "sum"])
ds_cclm = xr.merge([ds_cclm_tas, ds_cclm_pr])
del ds_cclm_tas, ds_cclm_pr


# Load the MAR data
## palceholder for MAR data -for plotting purposes
ds_mar = ds_alaro

#OBSERVATIONAL data

# Load CLIMATE_GRID
## Issue - currently can't load the ungridded CLIMATE_GRID data as there is no unique identifier for the data 

ds_ref = manager.load_data("CLIMATE_GRID", variables, path_identifiers=["regridded"])

# Create a DataTree object
data_dict = {
    "RCM/ERA5/ALARO1_SFX": ds_alaro,
    "RCM/ERA5/CCLM6-0-1-URB-ESG": ds_cclm,
    "RCM/ERA5/MAR": ds_mar,
    "obs/CLIMATE_GRID": ds_ref
}

dt = DataTree.from_dict(data_dict)

############################################
# STEP 2: Preprocessing the data

## Regrid (currently to CLIMATE_GRID)
dt["RCM"] = dt["RCM"].map_over_subtree(vp.remap_xesmf, dt.obs.CLIMATE_GRID.to_dataset(), method="bilinear", regridding_kwargs={"keep_attrs": True})

## Select the time period from 1980 to 2002 (inclusive)
dt = dt.sel(time=slice(f"{period[0]}-01-01", f"{period[1]}-12-31"))


############################################
# STEP 3: Diagnostics

In [ ]:
############################################
# STEP 3: Diagnostics

#Compute the data once (not for every diagnostic separately)
with ProgressBar():
    dt = dt.compute()


In [3]:
dt["RCM"] = dt["RCM"].map_over_subtree(vp.remap_xesmf, dt.obs.CLIMATE_GRID.to_dataset(), method="conservative", regridding_kwargs={"keep_attrs": True})

## Select the time period from 1980 to 2002 (inclusive)
dt = dt.sel(time=slice(f"{period[0]}-01-01", f"{period[1]}-12-31"))

KeyError: 'lon_b'

In [12]:

dt_timmean

DataTree('None', parent=None)
├── DataTree('RCM')
│   └── DataTree('ERA5')
│       ├── DataTree('ALARO1_SFX')
│       │       Dimensions:  (y: 483, x: 483)
│       │       Coordinates:
│       │         * x        (x) float64 4kB 0.0 12.5 25.0 37.5 ... 6e+03 6.012e+03 6.025e+03
│       │         * y        (y) float64 4kB 0.0 12.5 25.0 37.5 ... 6e+03 6.012e+03 6.025e+03
│       │           lon      (y, x) float64 2MB dask.array<chunksize=(483, 483), meta=np.ndarray>
│       │           lat      (y, x) float64 2MB dask.array<chunksize=(483, 483), meta=np.ndarray>
│       │           crs      |S1 1B b''
│       │       Data variables:
│       │           pr       (y, x) float32 933kB dask.array<chunksize=(483, 483), meta=np.ndarray>
│       │           tas      (y, x) float32 933kB dask.array<chunksize=(483, 483), meta=np.ndarray>
│       │       Attributes: (12/31)
│       │           activity_id:            DD
│       │           Conventions:            CF-1.11
│       │           driving_experiment:     reanalysis simulation of the recent past
│       │           driving_experiment_id:  evaluation
│       │           driving_institute_id:   ECMWF
│       │           driving_source_id:      ERA5
│       │           ...                     ...
│       │           endtime:                347151600.0
│       │           frequency:              day
│       │           creation_date:          2024-12-18T14:48:50Z
│       │           tracking_id:            hdl:21.14103/d83456bb-c23d-4195-9528-124b0acb4454
│       │           history:                Created by CordextractR v2.0.2
│       │           variable_id:            pr
│       ├── DataTree('CCLM6-0-1-URB-ESG')
│       │       Dimensions:    (rlat: 434, rlon: 446)
│       │       Coordinates:
│       │           lon        (rlat, rlon) float32 774kB dask.array<chunksize=(434, 446), meta=np.ndarray>
│       │           lat        (rlat, rlon) float32 774kB dask.array<chunksize=(434, 446), meta=np.ndarray>
│       │         * rlon       (rlon) float32 2kB -29.58 -29.48 -29.36 ... 19.15 19.25 19.36
│       │         * rlat       (rlat) float32 2kB -24.58 -24.48 -24.36 ... 22.83 22.93 23.05
│       │           height_2m  float32 4B 2.0
│       │       Data variables:
│       │           tas        (rlat, rlon) float32 774kB dask.array<chunksize=(338, 347), meta=np.ndarray>
│       │           pr         (rlat, rlon) float64 2MB dask.array<chunksize=(338, 347), meta=np.ndarray>
│       │       Attributes: (12/18)
│       │           CDI:               Climate Data Interface version 2.3.0 (https://mpimet.m...
│       │           Conventions:       CF-1.4
│       │           source:            cosmo6.00_clm2, int2lm3.0_clm1
│       │           institution:       KU Leuven, Leuven, BELGIUM
│       │           title:             CB2_CCLM_EUR11_ERA5_evaluation
│       │           project_id:        CORDEXbe2
│       │           ...                ...
│       │           history:           Thu Dec 19 17:10:57 2024: cdo daymean daily/temp/T_2M_...
│       │           frequency:         day
│       │           CDO:               Climate Data Operators version 2.3.0 (https://mpimet.m...
│       │           freq:              daily
│       │           path_identifiers:  ['CB2_CCLM_EUR11_ERA5_evaluation', 'mean']
│       │           dataset:           CCLM
│       └── DataTree('MAR')
│               Dimensions:  (y: 483, x: 483)
│               Coordinates:
│                 * x        (x) float64 4kB 0.0 12.5 25.0 37.5 ... 6e+03 6.012e+03 6.025e+03
│                 * y        (y) float64 4kB 0.0 12.5 25.0 37.5 ... 6e+03 6.012e+03 6.025e+03
│                   lon      (y, x) float64 2MB dask.array<chunksize=(483, 483), meta=np.ndarray>
│                   lat      (y, x) float64 2MB dask.array<chunksize=(483, 483), meta=np.ndarray>
│                   crs      |S1 1B b''
│               Data variables:
│                   pr       (y, x) float32 933kB dask.array<chunksize=(483, 483), meta=n

In [ ]:

## Plot maps

# settings
averaging_period = ["annual"] # JJA, DJF
variable = "tas"

# calculate time mean for every dataset in tree
def mean_over_time(ds):
    return ds.mean(dim='time')

dt_timmean = dt.map_over_subtree(mean_over_time)


# create figure 
fig, axes = plt.subplots(2,2,figsize=(15,10))
axes = axes.flatten()

for i, dataset_name in enumerate(list(data_dict.keys())):
    ax = axes[i]
    dt_timmean[dataset_name][variable].plot(ax=ax, cmap='coolwarm')


# Mean maps





[##################                      ] | 46% Completed | 130.26 s

In [ ]:
#Model2Ref
## Spatial Bias
from valenspy.diagnostic import SpatialBias
with ProgressBar():
    ds_spbias = SpatialBias(dt["RCM/ERA5/ALARO1_SFX"].to_dataset(), dt["obs/CLIMATE_GRID"].to_dataset())
    ds_spbias = ds_spbias.compute()

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
SpatialBias.plot(ds_spbias.tas, ax=ax[0])
SpatialBias.plot(ds_spbias.pr, ax=ax[1])
plt.savefig("CORDEX_eval_scripts/plots/Spatial_bias.png")

In [ ]:
from valenspy.diagnostic import SpatialBias
with ProgressBar():
    ds_spbias = SpatialBias(dt["RCM/ERA5/ALARO1_SFX"].to_dataset(), dt["obs/CLIMATE_GRID"].to_dataset())
    ds_spbias = ds_spbias.compute()
